In [1]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    #! mkdir -p ../data
    # los que usan colab deben modificar el token de esta url:
    #! wget -O ../data/properati_2017.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
Running command `conda install --yes chart-studio=1.1.0`... error
    
PackagesNotFoundError: The following packages are not available from current channels:

  - chart-studio=1.1.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.





In [2]:
import pandas as pd
import numpy as np
import re

In [6]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).
data = pd.read_csv("./data/properati_2017.csv", sep = ",", low_memory=False)

In [7]:
def drop_columns(df, columns):
    all_columns = set(df.columns)
    relevant_columns = all_columns.difference(set(columns))
    return df.loc[:, list(relevant_columns)]

data = data[data['currency'] == 'USD']
data = data[~data['price_usd_per_m2'].isna()]

surface_mask = data[['surface_total_in_m2', 'surface_covered_in_m2']].notna().all(axis=1)
data = data[surface_mask]

data = drop_columns(data, ['Unnamed: 0', 'image_thumbnail',
                           'properati_url', 'place_with_parent_names',
                           'lat-lon', 'properati_url', 'image_thumbnail',
                           'country_name', 'price_aprox_local_currency',
                           'price_aprox_usd', 'price_per_m2',
                           'description', 'title', 'currency'])

In [8]:
data.head()

,state_name,property_type,lon,price_usd_per_m2,floor,operation,surface_total_in_m2,place_name,geonames_id,surface_covered_in_m2,lat,price,rooms,expenses
0,Capital Federal,PH,-58.508839,1127.272727,NaN,sell,55.0,Mataderos,3430787.0,40.0,-34.661824,62000.0,NaN,NaN
2,Capital Federal,apartment,-58.522982,1309.090909,NaN,sell,55.0,Mataderos,3430787.0,55.0,-34.652262,72000.0,NaN,NaN
4,Buenos Aires Costa Atlántica,apartment,-57.549447,1828.571429,NaN,sell,35.0,Centro,3435548.0,35.0,-38.002626,64000.0,NaN,NaN
6,Bs.As. G.B.A. Zona Norte,PH,-58.521782,1226.415094,NaN,sell,106.0,Munro,3430511.0,78.0,-34.532957,130000.0,NaN,NaN
7,Capital Federal,apartment,-58.443362,3066.666667,NaN,sell,45.0,Belgrano,3436077.0,40.0,-34.559873,138000.0,NaN,NaN
